In [8]:
from enum import Enum, auto
from openai_function_call import OpenAISchema
from pydantic import Field,BaseModel

class Role(Enum):
    PRINCIPAL = auto()
    TEACHER = "teacher"
    STUDENT = auto()
    OTHER = auto()

class UserDetail(BaseModel):
    age: int
    name: str
    role: Role = Field(description="Correctly assign one of the predefined roles to the user.")

user = UserDetail(age=30, name="John Doe", role=Role.TEACHER)
print(user.role)  # 输出 Role.TEACHER

Role.TEACHER


In [9]:
class UserDetails(OpenAISchema):
    """
    Correctly extracted user information

    :param name: the user's full name
    :param age: age of the user
    """
    name: str = Field(..., description="User's full name")
    age: int

UserDetails.openai_schema

{'name': 'UserDetails',
 'description': "Correctly extracted user information\n\n:param name: the user's full name\n:param age: age of the user",
 'parameters': {'properties': {'name': {'description': "User's full name",
    'type': 'string'},
   'age': {'type': 'integer'}},
  'required': ['age', 'name'],
  'type': 'object'}}

In [18]:
import openai
from instructor import OpenAISchema
from pydantic import Field
openai.api_type = "azure"
openai.api_key = "0396650923ad40b880bf2a3ce3b80b9b"
openai.api_base = "https://myshell0.openai.azure.com"
openai.api_version = "2023-07-01-preview"
#@markdown ##Para
model="gpt-3.5-turbo-16k" #@param["gpt-3.5-turbo-0301","gpt-3.5-turbo-0613","gpt-3.5-turbo-16k"]
temperature = 0.0 #@param {type:"slider", min:0, max:2, step:0.1}
top_p = 1  #@param {type:"slider", min:0, max:2, step:0.1}
max_tokens = 60 #@param {type:"slider", min:0, max:500, step:1}
frequency_penalty = 0 #@param {type:"slider", min:0, max:2, step:0.1}
presence_penalty = 0 #@param {type:"slider", min:0, max:2, step:0.1}

class UserDetails(OpenAISchema):
    name: str = Field(..., description="User's full name")
    age: int


completion = openai.ChatCompletion.create(
    temperature=temperature,
    top_p=top_p,
    model=model,
    deployment_id="gpt-35-turbo-0613",
    functions=[UserDetails.openai_schema],
    function_call={"name": UserDetails.openai_schema["name"]},
    messages=[
        {"role": "system", "content": "Extract user details from my requests"},
        {"role": "user", "content": "My name is John Doe and I'm 30 years old."},
    ],
)

user = UserDetails.from_response(completion)
print(UserDetails.openai_schema)
print(user.name)  # Output: John Doe
print(user.age)   # Output: 30

{'name': 'UserDetails', 'description': 'Correctly extracted `UserDetails` with all the required parameters with correct types', 'parameters': {'properties': {'name': {'description': "User's full name", 'type': 'string'}, 'age': {'type': 'integer'}}, 'required': ['age', 'name'], 'type': 'object'}}
John Doe
30


In [20]:
import asyncio

async def foo():
    print("Start foo")
    await asyncio.sleep(1)
    print("End foo")
    
asyncio.run(foo())

RuntimeError: asyncio.run() cannot be called from a running event loop